Before writing code in attempt to predicintg the US Presidential Election winner it is important to understand the data. 
Collected Data (All data was sourced from Kaggle): 
- Historical Election Data (2020) 
- Election Polling (2024)
- US President Economic Indicators (1981 - 2021)
- Social Sentiment of Election Candidates (Twitter 2020)

Historically, the race for the presidential election has been amongst two winning parties \bold{Democratics} (D) and \bold{Republicans} (R). Correctly predicitng the election can be challenging, it is even more so when such predictions can be linked to making volatile financial decisions. 

Such decisions can be linked for decidiing : 
1. Market Stability  + Investor confidence -> Stock market reaction, investment startegies 
2. Secotor-Specific Imapcts ( Health, Energy, ...)
3. Regulatorty Changes, Corporate Tax Policies, Tariffs and Trade Policies 
4. Impact on consumptions (interest rates (mortages, loans, bonds))+ Job Market (umemployement rate)


Before creating ML and DeepLearning Models it is important to conduct Explaratory Data Analysis (EDA) to better understand the structure of the data you are working with, idenitfying relationships, patterns, outliers and anomalies, to ultimately imporve model performance and reliable results.


Penultimate step EDA: 
### 1) Data Cleaning/Restructure

In [54]:
import pandas as pd
pd.set_option('display.max_columns', 200)
import seaborn as sbn 
import matplotlib.pyplot as plt

1) US Economy President KPI ( Key Performance Indicator ) dataset

In [55]:
df_economy = pd.read_csv("Data/USA Economy Growth.csv", encoding='ISO-8859-1') 
df_economy.head()

# reorder the csv file columns for better reading
new_order = ['Presidents', 'Year'] + list(df_economy.columns[2:9])
df_economy = df_economy.reindex(columns = new_order) 

# Name does not matter, what matters is the party the president represents 
print(df_economy['Presidents'].describe())
names = df_economy['Presidents'].unique()
print(names)


df_economy['Presidents'] = df_economy['Presidents'].replace({'Ronald Reagan':'R', 'George Bush':'R', '':'D', 'George W. Bush':'R', 'Barack Obama':'D', 'Donald Trump':'R', 'Joe Biden':'D'})
df_economy.tail()

count                43
unique                7
top       Ronald Reagan
freq                  8
Name: Presidents, dtype: object
['Ronald Reagan' 'George Bush' 'Bill Clinton' 'George W. Bush'
 'Barack Obama' 'Donald Trump' 'Joe Biden']


,Presidents,Year,GDP per capita (in US$ PPP),GDP (in Bil. US$nominal),GDP per capita (in US$ nominal),GDP growth %,Inflation rate %,Unemployment %,Government debt (in % of GDP)
38,R,2019,65077.3,21381.0,65077.3,2.30%,1.80%,3.70%,108.80%
39,R,2020,63577.3,21060.5,63577.3,-2.80%,1.20%,8.10%,133.40%
40,D,2021,70159.8,23315.1,70159.8,5.90%,4.70%,5.40%,126.40%
41,D,2022,76348.5,25464.5,76348.5,2.10%,8.00%,3.60%,121.70%
42,D,2023,80034.6,26854.6,80034.6,1.60%,4.50%,3.80%,122.20%


2. Election Polling 

In [66]:
df_polling = pd.read_csv("Data/ElectionPollingData2024.csv", encoding = 'ISO-8859-1')

print(df_polling.shape)
print(df_polling.dtypes)
df_polling.describe()

(616, 41)
poll_id                      int64
pollster_id                  int64
pollster                    object
sponsor_ids                 object
sponsors                    object
display_name                object
pollster_rating_id           int64
pollster_rating_name        object
numeric_grade              float64
pollscore                  float64
methodology                 object
transparency_score         float64
state                      float64
start_date                  object
end_date                    object
sponsor_candidate_id       float64
sponsor_candidate          float64
sponsor_candidate_party    float64
question_id                  int64
sample_size                float64
population                  object
subpopulation               object
population_full             object
tracking                    object
created_at                  object
notes                       object
url                         object
source                     float64
internal  

,poll_id,pollster_id,pollster_rating_id,numeric_grade,pollscore,transparency_score,state,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,source,race_id,cycle,seat_number,dem,rep,ind
count,616.000000,616.000000,616.000000,576.000000,576.000000,239.000000,0.0,0.0,0.0,0.0,616.000000,608.000000,0.0,616.0,616.0,0.0,616.000000,616.000000,0.0
mean,84598.071429,1060.521104,280.863636,1.912674,-0.226910,6.083682,NaN,NaN,NaN,NaN,185224.313312,4064.430921,NaN,9549.0,2024.0,NaN,44.544903,43.834416,NaN
std,1564.734963,433.561353,166.227147,0.668623,0.736635,2.727718,NaN,NaN,NaN,NaN,9594.481417,3043.419823,NaN,0.0,0.0,NaN,2.017888,2.248153,NaN
min,81694.000000,194.000000,37.000000,0.500000,-1.300000,1.000000,NaN,NaN,NaN,NaN,165625.000000,500.000000,NaN,9549.0,2024.0,NaN,36.000000,35.000000,NaN
25%,83517.750000,568.000000,218.000000,1.800000,-0.600000,4.000000,NaN,NaN,NaN,NaN,179127.750000,1100.000000,NaN,9549.0,2024.0,NaN,43.000000,43.000000,NaN
50%,83671.500000,1189.000000,218.000000,1.800000,-0.200000,6.000000,NaN,NaN,NaN,NaN,179281.500000,2012.500000,NaN,9549.0,2024.0,NaN,45.000000,43.000000,NaN
75%,85911.000000,1189.000000,385.000000,2.400000,-0.200000,9.000000,NaN,NaN,NaN,NaN,191983.250000,7439.000000,NaN,9549.0,2024.0,NaN,46.000000,45.015000,NaN
max,87791.000000,1888.000000,861.000000,3.000000,1.700000,10.000000,NaN,NaN,NaN,NaN,206110.000000,8753.000000,NaN,9549.0,2024.0,NaN,52.000000,52.000000,NaN
